In [1]:
import numpy as np
import pandas as pd
import nltk

For text feature extraction

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

For splitting the data into training and test data

In [3]:
from sklearn.model_selection import train_test_split

Gaussian Naive-Bayes Classifier and Multinomial Naive-Bayes Classifier

In [4]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB

Random Forest Classifier

In [5]:
from sklearn.ensemble import RandomForestClassifier

Logistic Regression and Stochastic Gradient Descent (SGD) Classifier

In [6]:
from sklearn.linear_model import LogisticRegression, SGDClassifier

For determining classifier accuracy

In [7]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Import data preprocessor functions

In [8]:
from data_preprocessor import *

[nltk_data] Downloading package punkt to /Users/farhan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/farhan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/farhan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/farhan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /Users/farhan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Pickle library to save the trained model's state

In [9]:
import pickle

# Reading the Data
Opening the CSV file and reading it into a Pandas dataframe

In [10]:
dataset_good = read_file("processed_training_and_test_data/Hotel_Reviews_good.csv")
dataset_bad = read_file("processed_training_and_test_data/Hotel_Reviews_bad.csv")

print(len(dataset_good))
print(len(dataset_bad))

115758
10699


In [11]:
dataset_good.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,reviewContent,sentiment
0,7,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/17/2017,7.7,Hotel Arena,United Kingdom,Apart from the price for the brekfast Everyth...,11,1403,Good location Set in a lovely park friendly s...,19,1,10.0,"[' Leisure trip ', ' Couple ', ' Duplex Double...",17 days,52.360576,4.915968,Good location Set in a lovely park friendly s...,Positive
1,10,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/7/17,7.7,Hotel Arena,United Kingdom,Nothing all great,5,1403,Rooms were stunningly decorated and really sp...,101,2,10.0,"[' Leisure trip ', ' Group ', ' Duplex Double ...",27 days,52.360576,4.915968,Rooms were stunningly decorated and really sp...,Positive
2,15,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/3/17,7.7,Hotel Arena,Italy,No Negative,0,1403,This hotel is awesome I took it sincirely bec...,82,26,10.0,"[' Leisure trip ', ' Couple ', ' Duplex Double...",31 days,52.360576,4.915968,This hotel is awesome I took it sincirely bec...,Positive
3,48,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,3/22/2017,7.7,Hotel Arena,United Kingdom,No Negative,0,1403,The quality of the hotel was brilliant and ev...,77,3,10.0,"[' Leisure trip ', ' Couple ', ' Duplex Double...",134 day,52.360576,4.915968,The quality of the hotel was brilliant and ev...,Positive
4,52,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,3/18/2017,7.7,Hotel Arena,Netherlands,I loved everything,5,1403,The location in a quiet park with a great ter...,65,34,10.0,"[' Leisure trip ', ' Couple ', ' Duplex Twin R...",138 day,52.360576,4.915968,The location in a quiet park with a great ter...,Positive


In [ ]:
dataset_bad.head()

Each review sentiment category will have 10000 samples (This is due to not having enough bad reviews, and some reviews have to be used for test data)

In [12]:
dataset_good_sample = dataset_good.sample(n=10000)

In [13]:
dataset_bad_sample = dataset_bad.sample(n=10000)

In [14]:
dataset_good_sample.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,reviewContent,sentiment
1422,6795,1 Addington Street Lambeth London SE1 7RY Unit...,1322,10/31/2016,8.4,Park Plaza County Hall London,United Kingdom,Nothing,3,6117,Excellent location and knowledgable staff,6,2,10.0,"[' Leisure trip ', ' Family with young childre...",276 day,51.501400,-0.116009,Excellent location and knowledgable staff,Positive
72394,326420,George Gershwinlaan 101 Zuideramstel 1082 MT A...,203,7/19/2016,9.0,Crowne Plaza Amsterdam South,Philippines,Slow cleaning service,4,1349,Housekeeper Michael is excellent Very attenti...,18,2,10.0,"[' Business trip ', ' Group ', ' Superior Twin...",380 day,52.335733,4.874395,Housekeeper Michael is excellent Very attenti...,Positive
11292,48526,14 16 Craven Hill Lancaster Gate Westminster B...,289,2/16/2017,8.8,Park Grand London Lancaster Gate,Australia,The rooms were very small,6,1519,The beds were really comfy and staff were rea...,11,1,10.0,"[' Leisure trip ', ' Group ', ' Club Double Ro...",168 day,51.513207,-0.180782,The beds were really comfy and staff were rea...,Positive
58800,265965,Albany Street London NW1 3UP United Kingdom,354,8/18/2015,8.3,Melia White House Hotel,United Kingdom,Some staff couldn t understand english,7,1871,Lovely hotel wil stay here again,7,2,10.0,"[' Leisure trip ', ' Couple ', ' Classic Twin ...",716 day,51.529860,-0.144462,Lovely hotel wil stay here again,Positive
79101,356553,Keizersgracht 384 Amsterdam City Center 1016 G...,64,6/11/16,9.1,The Dylan Amsterdam,United States of America,No Negative,0,314,Great location and wonderful stsff,6,1,10.0,"[' Leisure trip ', ' Couple ', ' Double Room '...",418 day,52.365469,4.879871,Great location and wonderful stsff,Positive


Reset the index (As the sample is random, the index must be reset)

In [15]:
dataset_good_sample = dataset_good_sample.reset_index(drop=True)
print(f"Number of good reviews: {len(dataset_good_sample)}")

Number of good reviews: 10000


In [16]:
dataset_bad_sample = dataset_bad_sample.reset_index(drop=True)
print(f"Number of bad reviews: {len(dataset_bad_sample)}")

Number of bad reviews: 10000


Combine both positive and negative reviews to a dataframe

In [17]:
dataset = pd.concat([dataset_good_sample, dataset_bad_sample])
dataset["reviews"] = dataset["reviewContent"]

In [18]:
dataset.head()

,Unnamed: 0,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,...,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,reviewContent,sentiment,reviews
0,6795,1 Addington Street Lambeth London SE1 7RY Unit...,1322,10/31/2016,8.4,Park Plaza County Hall London,United Kingdom,Nothing,3,6117,...,6,2,10.0,"[' Leisure trip ', ' Family with young childre...",276 day,51.501400,-0.116009,Excellent location and knowledgable staff,Positive,Excellent location and knowledgable staff
1,326420,George Gershwinlaan 101 Zuideramstel 1082 MT A...,203,7/19/2016,9.0,Crowne Plaza Amsterdam South,Philippines,Slow cleaning service,4,1349,...,18,2,10.0,"[' Business trip ', ' Group ', ' Superior Twin...",380 day,52.335733,4.874395,Housekeeper Michael is excellent Very attenti...,Positive,Housekeeper Michael is excellent Very attenti...
2,48526,14 16 Craven Hill Lancaster Gate Westminster B...,289,2/16/2017,8.8,Park Grand London Lancaster Gate,Australia,The rooms were very small,6,1519,...,11,1,10.0,"[' Leisure trip ', ' Group ', ' Club Double Ro...",168 day,51.513207,-0.180782,The beds were really comfy and staff were rea...,Positive,The beds were really comfy and staff were rea...
3,265965,Albany Street London NW1 3UP United Kingdom,354,8/18/2015,8.3,Melia White House Hotel,United Kingdom,Some staff couldn t understand english,7,1871,...,7,2,10.0,"[' Leisure trip ', ' Couple ', ' Classic Twin ...",716 day,51.529860,-0.144462,Lovely hotel wil stay here again,Positive,Lovely hotel wil stay here again
4,356553,Keizersgracht 384 Amsterdam City Center 1016 G...,64,6/11/16,9.1,The Dylan Amsterdam,United States of America,No Negative,0,314,...,6,1,10.0,"[' Leisure trip ', ' Couple ', ' Double Room '...",418 day,52.365469,4.879871,Great location and wonderful stsff,Positive,Great location and wonderful stsff


Convert the reviews to a list, for data preprocessing

In [19]:
reviews = reviews_to_list(dataset)

# Data Preprocessing
The data preprocessing step consists of five major steps:

1. Tokenization: The process of taking each review, in the form of a sentence, and retrieving the words in each review. Also removes any punctuation in the review.
2. Normalizing (Lemmatizing): Reduce each word to its most basic form. For example, without the tag, the word __*leaves*__ would be lemmatized to __*leaf*__, whereas the tagged version would be lemmatized to __*leave*__.
3. Removing Noise: Removes stopwords and numbers.
4. Forming corpuses: All the normalised words in each review will be combined to a sentence called a corpus.
5. Feature extraction: Extract all the features from the review(s).

In [20]:
processed_reviews = [text_preprocessing(review) for review in reviews]

In [21]:
corpus = form_corpus(processed_reviews)

In [22]:
for i in range(5):
    print(corpus[i])
print('...')

excellent location knowledgable staff
housekeeper michael excellent attentive desk staff wonderful
bed comfy staff friendly
lovely hotel wil stay
great location wonderful stsff
...


In [23]:
vectorizer = vectorizer()

The numbers in the list of lists below indicate the occurrence of each feature (word) in the review. For example, the word __bed__ appears once in the second review (corpus), and hence, 1 appears in the second list (NOTE: Just an example of how the `fit_transform().toarray()` method works).

In [24]:
features = vectorizer.fit_transform(corpus).toarray()

print(features, end='\n\n')
print(len(features[0]))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]

12992


In [25]:
feature_df = pd.DataFrame(data=features, columns=vectorizer.get_feature_names_out())
feature_df.tail()

,aa,aaawas,aafter,aand,aarangements,aback,abad,abandon,abandoned,abbey,...,zita,zlatki,zoltan,zombie,zone,zoo,zore,ztaff,zuid,zusje
19995,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19999,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Highlights the most common features (words) used to describe a review

In [26]:
print(vectorizer.get_feature_names_out())

['aa' 'aaawas' 'aafter' ... 'ztaff' 'zuid' 'zusje']


In [27]:
labels = dataset['sentiment'].tolist()
len(labels)

20000

### Splitting the dataset
In this section, the `test_train_split()` function returns four items, the __training features__, the __training labels__, the __test features__ and the __test labels__.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.20, random_state=0)

In [ ]:
print(X_train)

## Training and Testing the Models
In this section, we will use the features and labels to train and test the models. In this application, we will experiment with four classifier algorithms, namely:

1. Gaussian Naive-Bayes Classifier
2. Random Forest Classifier
3. Logistic Regression Classifier
4. Stochastic Gradient Descent Classifier

In [ ]:
print(X_train.shape)
print(X_test.shape)

### Gaussian Naive-Bayes Classifier

In [ ]:
naive_bayes_classifier = GaussianNB()
naive_bayes_classifier.fit(X_train, y_train)

In [ ]:
naive_bayes_classifier_prediction = naive_bayes_classifier.predict(X_test)
print(naive_bayes_classifier_prediction)

In [ ]:
print(f"Accuracy: {accuracy_score(y_test, naive_bayes_classifier_prediction)}")

In [ ]:
print(classification_report(y_test, naive_bayes_classifier_prediction))

### Random Forest Classifier

In [ ]:
random_forest_classifier = RandomForestClassifier(n_estimators=250, random_state=42)
random_forest_classifier.fit(X_train, y_train)

In [ ]:
random_forest_classifier_prediction = random_forest_classifier.predict(X_test)
print(random_forest_classifier_prediction)

In [ ]:
print(f"Accuracy: {accuracy_score(y_test, random_forest_classifier_prediction)}")

In [ ]:
print(classification_report(y_test, random_forest_classifier_prediction))

In [ ]:
print(confusion_matrix(y_test, random_forest_classifier_prediction))

### Logistic Regression Classifier

In [ ]:
logistic_regression_classifier = LogisticRegression(max_iter=1000)
logistic_regression_classifier.fit(X_train, y_train)

In [ ]:
logistic_regression_classifier_prediction = logistic_regression_classifier.predict(X_test)
print(logistic_regression_classifier_prediction)

In [ ]:
print(f"Accuracy: {accuracy_score(y_test, logistic_regression_classifier_prediction)}")

In [ ]:
print(classification_report(y_test, logistic_regression_classifier_prediction))

In [ ]:
print(confusion_matrix(y_test, logistic_regression_classifier_prediction))

### Stochastic Gradient Descent Classifier (SGD)

In [ ]:
sgd_classifier = SGDClassifier(loss='modified_huber')
sgd_classifier.fit(X_train, y_train)

In [ ]:
sgd_classifier_prediction = sgd_classifier.predict(X_test)

In [ ]:
print(f"Accuracy: {accuracy_score(y_test, sgd_classifier_prediction)}")

In [ ]:
print(classification_report(y_test, sgd_classifier_prediction))

### Multinomial Naive-Bayes Classifier

In [ ]:
mnb_classifier = MultinomialNB()
mnb_classifier.fit(X_train, y_train)

In [ ]:
mnb_classifier_prediction = mnb_classifier.predict(X_test)

In [ ]:
print(f"Accuracy: {accuracy_score(y_test, mnb_classifier_prediction)}")

In [ ]:
print(classification_report(y_test, mnb_classifier_prediction))

In [ ]:
print(confusion_matrix(y_test, mnb_classifier_prediction))

## Save the trained models for future use

The trained models will be saved using Pickle.

In [ ]:
naive_bayes_file = open('trained_models/NaiveBayes', 'wb')
random_forest_file = open('trained_models/RandomForest', 'wb')
logistic_regression_file = open('trained_models/LogisticRegression', 'wb')
sgd_file = open('trained_models/SGD', 'wb')
mnb_file = open('trained_models/MNB', 'wb')

vectorizer_file = open('feature_extractor/vectorizer', 'wb')

pickle.dump(naive_bayes_classifier, naive_bayes_file)
pickle.dump(random_forest_classifier, random_forest_file)
pickle.dump(logistic_regression_classifier, logistic_regression_file)
pickle.dump(sgd_classifier, sgd_file)
pickle.dump(mnb_classifier, mnb_file)

pickle.dump(vectorizer, vectorizer_file)

naive_bayes_file.close()
random_forest_file.close()
logistic_regression_file.close()
sgd_file.close()
mnb_file.close()

vectorizer_file.close()